In [13]:
from __future__ import division, print_function
import numpy as np
import pandas as pd
# import scipy.stats as stats
# from scipy import stats
import stats

In [14]:
excel = pd.ExcelFile("Online Retail.xlsx")
df = excel.parse(excel.sheet_names[0])
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [15]:
df = df.iloc[:, 3:8]
df = df.assign(expenditure = df.Quantity * df.UnitPrice)
df.head()

,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,expenditure
0,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [16]:
df = df.assign(dow = df.InvoiceDate.dt.dayofweek.astype(np.int),
                 year = df.InvoiceDate.dt.strftime('%Y').astype(np.int),
                 month = df.InvoiceDate.dt.strftime('%m').astype(np.int),
                 day = df.InvoiceDate.dt.strftime('%d').astype(np.int))
df = df.assign(year = df.year - np.min(df.year), month = df.month - 1,
                 day = df.day - 1)
#Day of Week : Monday=0, Tuesday=1, ..., Friday=4, Saturday=5, Sunday=6
#              But, we will additionally normalize this column later because Saturday is missing.
#Year : 2010=0, 2011=1
#Month : January=0, February=1, ..., November=10, December=11
#Date : 1st=0, 2nd=1, ..., 30th=29, 31st=30
df.head()

,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,expenditure,dow,year,month,day
0,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,2,0,11,0
1,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2,0,11,0
2,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00,2,0,11,0
3,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2,0,11,0
4,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,2,0,11,0


In [17]:
df["CustomerID"] = df["CustomerID"].astype(str)
df = df.groupby(["CustomerID", "year", "month", "day", "dow"], as_index=False).agg({"expenditure":"sum", "Country":lambda x: stats.mode(x)})
df.head()

,CustomerID,year,month,day,dow,expenditure,Country
0,12346.0,1,0,17,1,0.00,United Kingdom
1,12347.0,0,11,6,1,711.79,Iceland
2,12347.0,1,0,25,2,475.39,Iceland
3,12347.0,1,3,6,3,636.25,Iceland
4,12347.0,1,5,8,3,382.52,Iceland


In [24]:
df = df.sort_values("Country")
df["Country"], list_country = pd.factorize(df["Country"])
df.head()
df.tail()

,CustomerID,year,month,day,dow,expenditure,Country
1454,327,1,6,27,3,546.43,37
13118,2935,1,5,19,0,185.78,37
54,16,1,7,21,0,252.90,37
53,16,1,3,10,0,299.10,37
6632,1425,1,7,21,0,278.13,37


In [19]:
for index, name in zip(set(df["Country"]), list_country):
    print(index, name)

0 Australia
1 Austria
2 Bahrain
3 Belgium
4 Brazil
5 Canada
6 Channel Islands
7 Cyprus
8 Czech Republic
9 Denmark
10 EIRE
11 European Community
12 Finland
13 France
14 Germany
15 Greece
16 Hong Kong
17 Iceland
18 Israel
19 Italy
20 Japan
21 Lebanon
22 Lithuania
23 Malta
24 Netherlands
25 Norway
26 Poland
27 Portugal
28 RSA
29 Saudi Arabia
30 Singapore
31 Spain
32 Sweden
33 Switzerland
34 USA
35 United Arab Emirates
36 United Kingdom
37 Unspecified


In [20]:
df = df.sort_values("CustomerID")
df["CustomerID"], list_ID = pd.factorize(df["CustomerID"])
df.head()

,CustomerID,year,month,day,dow,expenditure,Country
0,0,1,0,17,1,0.00,36
4,1,1,5,8,3,382.52,17
5,1,1,7,1,1,584.91,17
7,1,1,11,6,2,224.82,17
1,1,0,11,6,1,711.79,17


In [21]:
df = df.sort_values("dow")
df["dow"], list_dow = pd.factorize(df["dow"])
#Before : Monday=0, Tuesday=1, ..., Friday=4, Saturday=5, Sunday=6
#After : Monday=0, Tuesday=1, ..., Friday=4, Saturday=Missing, Sunday=5
for index, name in zip(set(df["dow"]), list_dow):
    print(index, name)

0 0
1 1
2 2
3 3
4 4
5 6


In [28]:
df.head()

,CustomerID,year,month,day,dow,expenditure,Country
101,33,0,11,7,2,258.90,0
270,72,1,2,23,3,286.25,0
257,69,1,9,5,3,678.76,0
108,34,1,10,23,3,286.40,0
106,34,1,6,13,3,902.09,0


In [27]:
df.tail()

,CustomerID,year,month,day,dow,expenditure,Country
1454,327,1,6,27,3,546.43,37
13118,2935,1,5,19,0,185.78,37
54,16,1,7,21,0,252.90,37
53,16,1,3,10,0,299.10,37
6632,1425,1,7,21,0,278.13,37
